In [ ]:
# from scipy.sparse import csr_matrix

### Входные данные

#### Загрузка

In [ ]:
# !unzip capstone_user_identification.zip

In [12]:
# from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import pickle

In [13]:
# PATH_TO_DATA = '../capstone_user_identification'
PATH_TO_DATA = '../input/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2'

times = ['time%s' % i for i in range(1, 11)]
# sites = ['site%s' % i for i in range(1, 11)]
df_train = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions.csv'), index_col='session_id', parse_dates=times)
df_test = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_sessions.csv'), index_col='session_id', parse_dates=times)
df_train.shape, df_test.shape

((253561, 21), (82797, 20))

In [14]:
df_train.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,2014-02-20 10:02:45,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,...,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0
3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,2013-12-16 16:40:19,...,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,0
4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,2014-03-28 10:54:12,...,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42,0
5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,2014-02-28 10:55:23,...,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11,0


In [15]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253561 entries, 1 to 253561
Data columns (total 21 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   site1   253561 non-null  int64         
 1   time1   253561 non-null  datetime64[ns]
 2   site2   250098 non-null  float64       
 3   time2   250098 non-null  datetime64[ns]
 4   site3   246919 non-null  float64       
 5   time3   246919 non-null  datetime64[ns]
 6   site4   244321 non-null  float64       
 7   time4   244321 non-null  datetime64[ns]
 8   site5   241829 non-null  float64       
 9   time5   241829 non-null  datetime64[ns]
 10  site6   239495 non-null  float64       
 11  time6   239495 non-null  datetime64[ns]
 12  site7   237297 non-null  float64       
 13  time7   237297 non-null  datetime64[ns]
 14  site8   235224 non-null  float64       
 15  time8   235224 non-null  datetime64[ns]
 16  site9   233084 non-null  float64       
 17  time9   233084 non-null  date

Словарь сайтов

In [16]:
with open(os.path.join(PATH_TO_DATA, 'site_dic.pkl'), 'rb') as f:
    site2id  = pickle.load(f)
id2site = {v:k for (k, v) in site2id.items()}
id2site[0] = 'unknown'

### Обработка

In [17]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler

import numpy as np

In [18]:
df_train.sort_values(by='time1', inplace=True)

In [19]:
class DataPreparator(BaseEstimator, TransformerMixin):
    """
    Fill NaN with zero values;
    Prepare a (Count)Vectorizer friendly 2D-list from data;
    Site ids to names.
    """
    def __init__(self, id2site):
        super().__init__()
        self.id2site = id2site
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        sites = ['site%s' % i for i in range(1, 11)]
        # Convert dataframe rows to strings
        return X[sites].fillna(0).astype('int').apply(lambda row: ' '.join([self.id2site[i] for i in row]), axis=1).tolist()

In [20]:
class ListPreparator(BaseEstimator, TransformerMixin):
    """
    Prepare a CountVectorizer friendly 2D-list from data.
    """
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X = X.values.tolist()
        # Convert dataframe rows to strings
        return [" ".join([str(site) for site in row]) for row in X]

In [10]:
# vectorizer_params = {
#     'ngram_range': (1, 5),
#     'max_features': 50000,
#     'tokenizer': lambda s: s.split()
# }

### Feature engineering

In [21]:
from scipy.sparse import hstack

In [49]:
class AttributesAdder(BaseEstimator, TransformerMixin):
    """
    Add new attributes to training and test set.
    """ 
    def fit(self, X, y=None):
        return self 
    
    def transform(self, X, y=None):
        # intraday features
        hour = X['time1'].apply(lambda ts: ts.hour)#.values
        morning = ((hour >= 7) & (hour <= 11)).astype('int')#.reshape(-1, 1)
        day = ((hour >= 12) & (hour <= 18)).astype('int')
        evening = ((hour >= 19) & (hour <= 23)).astype('int')
#         night = ((hour >= 0) & (hour <= 6)).astype('int')
        
        # season features
        month = X['time1'].apply(lambda ts: ts.month)
        summer = ((month >= 6) & (month <= 8)).astype('int')
        
        # day of the week features
        weekday = X['time1'].apply(lambda ts: ts.weekday()).astype('int')
        #     is_monday = np.isin(weekday, [0])
        #     is_wednesday = np.isin(weekday, [2])
        #     is_sunday = np.isin(weekday, [6])
        #     is_weekend = np.isin(weekday, [0, 2, 6])
        #     weekdays = pd.get_dummies(weekday.flatten()).to_numpy().reshape(7, -1, 1)
        
        # year features
        year = X['time1'].apply(lambda ts: ts.year).astype('int')
#         year = df['time1'].apply(lambda t: 100 * t.year).values.reshape(-1, 1) / 1e5
#         year_month = df['time1'].apply(lambda t: 100 * t.year + t.month).values.reshape(-1, 1) / 1e5
        
        X = np.c_[morning.values, day.values, evening.values, weekday.values, year.values]
        # summer.values, year.values
        return X

In [ ]:
def features(df):    
    # session features
    duration = (df[times].max(axis=1) - df[times].min(axis=1))\
                    .astype('timedelta64[ms]').astype(int).values.reshape(-1, 1)
#     unique_sites = df[sites].apply(lambda sites: sites.nunique(), axis=1).values.reshape(-1, 1)
#     number_of_sites = df[times].isnull().sum(axis=1).apply(lambda x: 10 - x).values.reshape(-1, 1)
#     time_per_site = (duration / number_of_sites)
    duration = scaler.transform(duration)
#     scaled = scaler.transform(np.concatenate((duration, time_per_site), axis=1)).reshape(2, -1, 1)
    
    # 
    feature_names =  ['morning', 'day', 'evening', 'year', 'year_month', 'weekday', 'month', 
                      'duration'] # + list(calendar.day_name)
    result = np.array([morning, day, evening, year, year_month, weekday, month, duration]) # is_wednesday, is_monday, is_sunday
#     result = np.concatenate((result, scaled), axis=0)
    
    return result, feature_names

In [23]:
class ScaledAttributesAdder(BaseEstimator, TransformerMixin):
    """
    Add new features, that should be scaled.
    """
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        # session time features
        times = ['time%s' % i for i in range(1, 11)]
        # session duration: take to the power of 1/5 to normalize the distribution
        session_duration = (X[times].max(axis=1) - X[times].min(axis=1)).astype('timedelta64[ms]').astype(int) ** 0.2
        # number of sites visited in a session
        number_of_sites = X[times].isnull().sum(axis=1).apply(lambda x: 10 - x)
        # average time spent on one site during a session
        time_per_site = (session_duration / number_of_sites) ** 0.2
        
        X = np.c_[session_duration.values]
        return X

In [50]:
feature_names = ['morning', 'day', 'evening', 'weekday', 'hour', 'year']

In [51]:
vectorizer_pipeline = Pipeline([
    ("preparator", DataPreparator(id2site)),
#     ("list_preparator", ListPreparator()),
    ("vectorizer", CountVectorizer(ngram_range=(1, 2)))
#     ("vectorizer", TfidfVectorizer(ngram_range=(1, 3), max_features=50000))
])

attributes_pipeline = Pipeline([
    ("adder", AttributesAdder())
])

# scaled_attributes_pipeline = Pipeline([
#     ("adder", ScaledAttributesAdder()),
#     ("scaler", StandardScaler())
# ])

In [44]:
%%time
X_train_vectorized = vectorizer_pipeline.fit_transform(df_train)

CPU times: user 20 s, sys: 131 ms, total: 20.1 s
Wall time: 20.1 s


In [45]:
X_train_vectorized

<253561x129397 sparse matrix of type '<class 'numpy.int64'>'
	with 6665433 stored elements in Compressed Sparse Row format>

In [27]:
y_train = df_train["target"].astype('int').values

In [52]:
feature_pipeline = FeatureUnion(transformer_list=[
    ('attributes_pipeline', attributes_pipeline),
#     ('scaled_attributes_pipeline', scaled_attributes_pipeline)
])

In [53]:
%%time
X_train_features = feature_pipeline.fit_transform(df_train)

CPU times: user 6.61 s, sys: 5.12 ms, total: 6.61 s
Wall time: 6.63 s


In [54]:
X_train = hstack([X_train_vectorized, X_train_features])

In [56]:
X_train

<253561x129402 sparse matrix of type '<class 'numpy.int64'>'
	with 7385603 stored elements in COOrdinate format>

In [ ]:
# scaler = StandardScaler()
# duration_train = (train_df[times].max(axis=1) - train_df[times].min(axis=1))\
#                     .astype('timedelta64[ms]').astype(int).values.reshape(-1, 1)
# # number_of_sites = train_df[times].isnull().sum(axis=1).apply(lambda x: 10 - x).values.reshape(-1, 1)
# # time_per_site = (duration_train / number_of_sites)
# # scaler.fit(np.concatenate((duration_train, time_per_site), axis=1))
# scaler.fit(duration_train)

In [ ]:
# pd.DataFrame(X_train_features, columns=feature_names)

### Модель

In [28]:
from sklearn.model_selection import cross_val_score, TimeSeriesSplit
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

#### Обучение и подбор гиперпараметров

In [57]:
model = SGDClassifier(loss='log', random_state=2179, n_jobs=-1)
time_split = TimeSeriesSplit(n_splits=10)

In [ ]:
# grid = {
#     'penalty': ['l2'],
#     'alpha': np.linspace(3e-05, 5e-05, 11)
# }

In [ ]:
# %%time

# search = GridSearchCV(model, param_grid=grid, cv=time_split, scoring='roc_auc')
# search.fit(X_train, y_train)

In [ ]:
# search.best_params_, search.best_score_

In [58]:
%%time

# sites = ['site%s' % i for i in range(1, 11)]

cv_scores = cross_val_score(model, X_train, y_train, cv=time_split, scoring='roc_auc', n_jobs=-1)

CPU times: user 393 ms, sys: 72.1 ms, total: 465 ms
Wall time: 12.4 s


In [59]:
cv_scores, cv_scores.mean(), cv_scores.std()

(array([0.61144867, 0.46570207, 0.76037139, 0.70623399, 0.73000612,
        0.79660116, 0.78643702, 0.86052069, 0.7942028 , 0.8920979 ]),
 0.7403621810294394,
 0.11805741904859296)

0.9315069358401142,
 0.044345068769381366 0 2 6

0.9180564468510812,
 0.05889198351691508 baseline

0.9232962337585114,
 0.058257317090901546

0.9287111177172779,
 0.04434090003756869 2

### Тест

In [ ]:
import eli5
import calendar

def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                               
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [ ]:
%%time
X_test_vectorized = vectorizer_pipeline.transform(df_test)
X_test_features = feature_pipeline.transform(df_test)
X_test = hstack([X_test_vectorized, X_test_features])

In [ ]:
search.best_estimator_.fit(X_train, y)
logit_test_pred_proba = search.best_estimator_.predict_proba(X_test)

In [ ]:
pd.DataFrame({'feature': feature_names, 'coef': search.best_estimator_.coef_.flatten()[-len(feature_names):]})

In [ ]:
eli5.show_weights(estimator=search.best_estimator_, feature_names=vectorizer.get_feature_names() + feature_names, top=30)

In [ ]:
write_to_submission_file(logit_test_pred_proba[:, 1], 'result.csv')